# This page can be used to record voice dataset.

- upload `finetuning-dataset.csv` to your google drive in a folder called `speech-finetuning-files` (you will [find here](https://github.com/TheSoundOfAIOSR/rg_speech_to_text/tree/main/data/finetuning-dataset))
- assign your PERSONAL_PREFIX as you see below. Should be short, one or two character and a -. That will be the prefix of your audio files.
- record all the audio files. you should see the audio files in your google dirve in `speech-finetuning-files` folder.
- share your `speech-finetuning-files` folder with the person wo will continue to merge audio files from all the members.

In [79]:
PERSONAL_PREFIX = 'XY-'

In [80]:
!pip install ffmpeg-python

In [81]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [82]:
import numpy as np
import soundfile as sf

def record(id):
  audio, sr = get_audio()
  sf.write(id+'.wav', audio, sr)

def next_recording(dataset, index):
  if index >= len(dataset):
    print("The recording is finished")
    return index
  selected = dataset[index]
  print(selected[0], 'Read: ', selected[1])
  record(selected[0])
  sr, duration = decode_sample_rate_and_time(selected[0] + '.wav')
  print("recorded", selected[0], "of", duration, "seconds")
  return index + 1

In [83]:
def decode_sample_rate_and_time(audio_filename):
  """
  decode_sample_rate_and_time from a wav file
  return a tuple of framerate (Hz) and length of the audio (seconds)
  """
  from scipy.io import wavfile
  sr, data = wavfile.read(audio_filename)
  return sr, len(data)/sr

In [84]:
def plot_audio(audio):
  import matplotlib.pyplot as plt
  plt.figure(figsize=(20,10))
  plt.plot(audio)
  plt.show()

In [85]:
def load_dataset(prefix):
  d = np.loadtxt('finetuning-dataset.csv',delimiter='|',dtype=str)
  d[:, 0] = [''.join((prefix, x)) for x in d[:, 0]]
  return d

Mounting my google drive





In [86]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [87]:
curr_dir =  "/content/gdrive/My Drive/speech-finetuning-files/"
%cd "$curr_dir"

/content/gdrive/My Drive/speech-finetuning-files


Here you can try the recording. You can check also the form of the signal, including amplitude

In [ ]:
audio, sr = get_audio()

In [ ]:
 plot_audio(audio)

In [88]:
d = load_dataset(PERSONAL_PREFIX)
# set/reset the array indexes
arrayIndex = 0  # from this index we start

Record one by one, by successively calling the code line below

In [ ]:
arrayIndex = next_recording(d, arrayIndex)